Choose the correct environment with all the dependencies install

In [2]:
#import all dependencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import pyvista as pv 
import pandas as pd
import trimesh
from scipy.spatial import KDTree
import os

Thickness calculation and visualization of a single design

In [6]:
# Load the volume mesh file
vtk_file_path = "/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_volmesh_(vtk)/9.vtk" 
volume_mesh = pv.read(vtk_file_path)


# Load the surface mesh file
obj_file_path = "/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_surfmesh_(obj)/9.obj"
surface_mesh = trimesh.load_mesh(obj_file_path)


def calculate_thickness(volume_mesh, surface_mesh):
    # Extract point cloud
    point_cloud = volume_mesh.points  
    # Extract surface points
    surface_points = surface_mesh.vertices 
    # Compute nearest distance using KDTree
    kdtree = KDTree(surface_points)
    distances, _ = kdtree.query(point_cloud)
    thickness = np.quantile(distances, 0.75)
    return thickness, distances

thickness, distances = calculate_thickness(volume_mesh, surface_mesh)
# Assign distances as scalar values
volume_mesh["Distance"] = distances  
#make a planar cut on the mesh along the x-axis to view a crosssection 
volume_mesh = volume_mesh.clip('x', invert=True)
#Display the estimated thickness
print("Median Thickness:", thickness)
# Plot the point cloud with distances
plotter = pv.Plotter()
plotter.add_mesh(volume_mesh, scalars="Distance", cmap="jet", point_size=3, render_points_as_spheres=True)
#plotter.add_mesh(pv.PolyData(surface_points), color="white", point_size=3, render_points_as_spheres=True, opacity=0)
#plotter.add_scalar_bar(title="Distance to Surface")
plotter.show()

Median Thickness: 1.3150820182391025


Widget(value='<iframe src="http://localhost:55908/index.html?ui=P_0x2a15339b0_3&reconnect=auto" class="pyvista…

Automate the process to calculate the thickness for all designs and store them in a dataframe as a column in the original metadata

In [7]:
#load metadata
data = pd.read_csv("/Users/User/Documents/Project_JEB/JEB_Data/workingdata.csv")
thickness = []
#extract 'id' from the metadata set and access the corresponding volume and surface meshes
for part_id in data['id']:
    # Load the VTK volume mesh
    vtk_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_volmesh_(vtk)/{part_id}.vtk"  # Update with your actual file path
    volume_mesh = pv.read(vtk_file_path)


    # Load the surface mesh
    obj_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_surfmesh_(obj)/{part_id}.obj"
    surface_mesh = trimesh.load_mesh(obj_file_path)
    
    #calculate stiffness
    est_thickness, distances = calculate_thickness(volume_mesh, surface_mesh)
    thickness.append(est_thickness)

# Add the curvature column to the DataFrame
data['thickness'] = thickness

# Save the updated data
updated_data_path = '/Users/User/Documents/Project_JEB/JEB_Data/workingdata.csv'
data.to_csv(updated_data_path, index=False)